# RAG LangChain

In [1]:
!pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap datasets langchain-community ragatouille

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
cud

In [2]:
%reload_ext dotenv
%dotenv

cannot find .env file


In [3]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

pd.set_option(
    "display.max_colwidth", None
)  # This will be helpful when visualizing retriever outputs

### Load your knowledge base

In [4]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
path = '/kaggle/input/nursing-staff-information-2'
loader = DirectoryLoader(path, glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [6]:
# Number of imported pages
print("Number of imported pages:", len(documents))

Number of imported pages: 2628


In [7]:
# Example page
print("Example page: \n", documents[0])

Example page: 
 page_content='' metadata={'source': '/kaggle/input/nursing-staff-information-2/Taschenwissen Pflege Arzneimittel- Schnell - sicher - -- James Corbett -- 2021 -- Elsevier Health Science -- 9783437096891 -- def44bfa8a8faf54102bd77cf0340a03 -- Annas Archive.pdf', 'page': 0}


In [8]:
from nltk.tokenize import word_tokenize

def get_word_counts(document_collection):
    """Calculate the word count for each document in a collection."""
    word_counts = []

    for document in document_collection:
        text_content = document.page_content  # Extract text from the document
        words = word_tokenize(text_content)  # Tokenize the text into words
        word_count = len(words)  # Count the words
        word_counts.append(word_count)  # Append the count to the list

    return word_counts

In [9]:
# Count words per page BEFORE chunking
word_counts_before = get_word_counts(documents)
print(word_counts_before[0:10])

[0, 7, 20, 24, 27, 198, 266, 46, 1, 64]


In [10]:
# Splitting the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=128)
texts = text_splitter.split_documents(documents)

In [11]:
print("Number of chunks:", len(texts))

Number of chunks: 68835


In [12]:
print(texts[3])

page_content='Antitussiva\nExpektoranzien u. Sekretolytika\nNiere und Harnsystem\nDiuretika\nMagen-Darm-System\nAntiemetika\nUlkustherapeutika\nAntidiarrhoika\nLaxanzien\nSpasmolytika' metadata={'source': '/kaggle/input/nursing-staff-information-2/Taschenwissen Pflege Arzneimittel- Schnell - sicher - -- James Corbett -- 2021 -- Elsevier Health Science -- 9783437096891 -- def44bfa8a8faf54102bd77cf0340a03 -- Annas Archive.pdf', 'page': 3}


In [13]:
# Count words per page AFTER chunking
word_counts_after = get_word_counts(texts)
print(word_counts_after[0:10])

[7, 20, 21, 14, 24, 16, 39, 36, 44, 50]


In [14]:
# Plot the word count distribution BEFORE and AFTER chunking

import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Before Chunking", "After Chunking"))

trace1 = go.Histogram(x=word_counts_before, name='Before')
fig.add_trace(trace1, row=1, col=1)
trace2 = go.Histogram(x=word_counts_after, name='After')
fig.add_trace(trace2, row=1, col=2)

fig.update_layout(height=600, width=1200, title_text="Word Count Distribution: Before and After Chunking")
fig.show()

In [28]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "LeoLM/leo-mistral-hessianai-7b-chat"

#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_compute_dtype=torch.bfloat16,
#)
model = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME, #quantization_config=bnb_config,     
    torch_dtype = torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning:

TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()



generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [117]:
READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=2048,
)

# 1. Retriever - embeddings 🗂️
The __retriever acts like an internal search engine__: given the user query, it returns a few relevant snippets from your knowledge base.

In [15]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

#text_splitter = RecursiveCharacterTextSplitter(
#    chunk_size=256,  # The maximum number of characters in a chunk: we selected this value arbitrarily
#    chunk_overlap=54,  # The number of characters to overlap between chunks
#    add_start_index=True,  # If `True`, includes chunk's start index in metadata
#    strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
#)

#docs_processed = []
#for doc in RAW_KNOWLEDGE_BASE:
#    docs_processed += text_splitter.split_documents([doc])

We also have to keep in mind that when embedding documents, we will use an embedding model that accepts a certain maximum sequence length `max_seq_length`.

So we should make sure that our chunk sizes are below this limit because any longer chunk will be truncated before processing, thus losing relevancy.

In [16]:
docs_processed = texts

In [17]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter
print(
    f"Model's maximum sequence length: {SentenceTransformer('thenlper/gte-small').max_seq_length}"
)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-small")
#lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(docs_processed)]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model's maximum sequence length: 512


100%|██████████| 68835/68835 [00:17<00:00, 3873.07it/s]


➡️ Now the chunk length distribution looks better!

### 1.2 Building the vector database

We want to compute the embeddings for all the chunks of our knowledge base: to learn more about sentence embeddings, we recommend reading [this guide](https://osanseviero.github.io/hackerllama/blog/posts/sentence_embeddings/).

#### How does retrieval work?

Once the chunks are all embedded, we store them in a vector database. When the user types in a query, it gets embedded by the same model previously used, and a similarity search returns the closest documents from the vector database.

The technical challenge is thus, given a query vector, to quickly find the nearest neighbors of this vector in the vector database. To do this, we need to choose two things: a distance, and a search algorithm to find the nearest neighbors quickly within a database of thousands of records.

##### Nearest Neighbor search algorithm

There are plentiful choices for the nearest neighbor search algorithm: we go with Facebook's [FAISS](https://github.com/facebookresearch/faiss) since FAISS is performant enough for most use cases, and it is well known and thus widely implemented.

##### Distances

Regarding distances, you can find a good guide [here](https://osanseviero.github.io/hackerllama/blog/posts/sentence_embeddings/#distance-between-embeddings). In short:

- **Cosine similarity** computes the similarity between two vectors as the cosinus of their relative angle: it allows us to compare vector directions regardless of their magnitude. Using it requires normalizing all vectors, to rescale them into unit norm.
- **Dot product** takes into account magnitude, with the sometimes undesirable effect that increasing a vector's length will make it more similar to all others.
- **Euclidean distance** is the distance between the ends of vectors.

You can try [this small exercise](https://developers.google.com/machine-learning/clustering/similarity/check-your-understanding) to check your understanding of these concepts. But once vectors are normalized, [the choice of a specific distance does not matter much](https://platform.openai.com/docs/guides/embeddings/which-distance-function-should-i-use).

Our particular model works well with cosine similarity, so choose this distance, and we set it up both in the Embedding model, and in the `distance_strategy` argument of our FAISS index. With cosine similarity, we have to normalize our embeddings.

🚨👇 The cell below takes a few minutes to run on A10G!

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

EMBEDDING_MODEL_NAME = "thenlper/gte-small"

In [24]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

In [135]:
QUESTION = "Mein Blutdruckmessgerät ist ausgefallen. Kannst du mir die korrekte Vorgehensweise zur manuellen Blutdruckmessung erläutern?"

In [136]:
# Embed a user query in the same space
user_query = QUESTION
query_vector = embedding_model.embed_query(user_query)

In [137]:
print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
print(
    "\n==================================Top document=================================="
)
print(retrieved_docs[0].page_content)
print("==================================Metadata==================================")
print(retrieved_docs[0].metadata)


Starting retrieval for user_query='Mein Blutdruckmessgerät ist ausgefallen. Kannst du mir die korrekte Vorgehensweise zur manuellen Blutdruckmessung erläutern?'...

==================================Top document==================================
traarterielle Blutdruckmessung ange-
legt. Diese Maßnahme erfolgt idealer-
weise noch im wachen Zustand des Pa-
tienten. Außer einem Druck und einem
leichten Brennen, verursacht durch das
Lokalanästhetikum, wird Frau Schulz kei-
==================================Metadata==================================
{'source': '/kaggle/input/nursing-staff-information-2/Thiemes Pflege- Das Lehrbuch fur Pflegende in der Ausbildung  -  2020  -  Georg Thieme Verlag  -  9783132437685  -  b07ae6a4e31ef88daee4440765662459  -  Annas Archive_compressed (1)-1.pdf', 'page': 715}


# 2. Reader - LLM 💬

### 2.1. Reader model

The choice of a reader model is important in a few aspects:
- the reader model's `max_seq_length` must accommodate our prompt, which includes the context output by the retriever call: the context consists of 5 documents of 512 tokens each, so we aim for a context length of 4k tokens at least.
- the reader model

For this example, we chose [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), a small but powerful model.

With many models being released every week, you may want to substitute this model to the latest and greatest. The best way to keep track of open source LLMs is to check the [Open-source LLM leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

To make inference faster, we will load the quantized version of the model:

In [138]:
READER_LLM("Was ist 4+4? Antwort:")

[{'generated_text': ''}]

### 2.2. Prompt

The RAG prompt template below is what we will feed to the Reader LLM: it is important to have it formatted in the Reader LLM's chat template.

We give it our context and the user's question.

In [139]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Answer in the language of the question.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|im_start|>system
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Answer in the language of the question.<|im_end|>
<|im_start|>user
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}<|im_end|>
<|im_start|>assistant



Let's test our Reader on our previously retrieved documents!

In [142]:
retrieved_docs_text = [
    doc.page_content for doc in retrieved_docs
]  # We only need the text of the documents
context = "\nExtracted documents:\n"
context += "".join(
    [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)]
)

final_prompt = RAG_PROMPT_TEMPLATE.format(
    question=QUESTION, context=context
)

# Redact an answer
answer_full = READER_LLM(final_prompt)[0]
print(answer_full)
answer = answer_full["generated_text"]
print(answer)

{'generated_text': 'Antwort:\nWenn Ihr Blutdruckmessgerät ausgefallen ist, kann eine manuelle Blutdruckmessung zu Hause durchgeführt werden. Es gibt verschiedene Methoden zur Messung des Blutdrucks, aber die häufigste Methode ist die Messung mit einem Blutdruckmessgerät. Wenn Sie Ihr Gerät nicht verwenden können, sollten Sie einen Arzt aufsuchen, um Ihren Blutdruck zu überprüfen. Wenn Sie jedoch Ihren Blutdruck zu Hause manuell messen müssen, sollten Sie folgende Schritte befolgen:\n\n1. Suchen Sie eine gute Lichtquelle, damit Sie Ihren Arm und Ihren Finger gut sehen können.\n2. Waschen Sie Ihre Hände und Ihren Arm und trocknen Sie sie ab.\n3. Finden Sie eine Vene an Ihrem Arm, die flach liegt (d. h. nicht hervorsteht). Das ist wichtig, weil es dem Stethoskop hilft, den Puls zu hören.\n4. Legen Sie Ihren Finger unterhalb und parallel zum Handgelenkknochen.\n5. Drücken Sie Ihren Finger gegen den Handgelenkknochen, bis kein Puls mehr zu spüren ist. Dies ist der obere Bereich des systolis

In [143]:
for doc in retrieved_docs:
    print(doc.metadata)

{'source': '/kaggle/input/nursing-staff-information-2/Thiemes Pflege- Das Lehrbuch fur Pflegende in der Ausbildung  -  2020  -  Georg Thieme Verlag  -  9783132437685  -  b07ae6a4e31ef88daee4440765662459  -  Annas Archive_compressed (1)-1.pdf', 'page': 715}
{'source': '/kaggle/input/nursing-staff-information-2/Thiemes Pflege- Das Lehrbuch fur Pflegende in der Ausbildung  -  2020  -  Georg Thieme Verlag  -  9783132437685  -  b07ae6a4e31ef88daee4440765662459  -  Annas Archive_compressed (1)-1.pdf', 'page': 530}
{'source': '/kaggle/input/nursing-staff-information-2/Thiemes Pflege- Das Lehrbuch fur Pflegende in der Ausbildung  -  2020  -  Georg Thieme Verlag  -  9783132437685  -  b07ae6a4e31ef88daee4440765662459  -  Annas Archive_compressed (1)-1.pdf', 'page': 713}
{'source': '/kaggle/input/nursing-staff-information-2/Thiemes Pflege- Das Lehrbuch fur Pflegende in der Ausbildung  -  2020  -  Georg Thieme Verlag  -  9783132437685  -  b07ae6a4e31ef88daee4440765662459  -  Annas Archive_compress

## Simplification

In [144]:
level_examples = {
    "A1": [
        {
            "original": "»Rapunzel, Rapunzel, laß dein Haar herunter.« Denselben Tag aber, wo sie Rapunzel verstoßen hatte, machte abends die Zauberin die abgeschnittenen Flechten oben am Fensterhaken fest, und als der Königssohn kam und rief:",
            "simplified": "Rapunzel: Lass deine Haare herunter!"
        },
        {
            "original": "Da schliefen auch die Pferde im Stall, die Hunde im Hofe, die Tauben auf dem Dache, die Fliegen an der Wand, ja, das Feuer, das auf dem Herde flackerte, ward still und schlief ein, und der Braten hörte auf zu brutzeln, und der Koch, der den Küchenjungen, weil er etwas versehen hatte, in den Haaren ziehen wollte, ließ ihn los und schlief.",
            "simplified": "Und die Fliegen an der Wand. Die Tauben auf dem Dach. Die Pferde im Stall."
        },
        {
            "original": "JHWH vergelte dein Tun und dein Lohn sei voll von JHWH, dem Gott Israels, zum du gekommen bist, um unter seinen Flügeln Schutz zu suchen! “ Sie antwortete: „Möge ich in deinen Augen Gefallen finden, mein Herr, weil du mich getröstet und weil du zum Herzen deiner Magd gesprochen hast.",
            "simplified": "Gott wird dich beschenken. Denn du bist gut. Gott wird dich beschützen."
        }
    ],
    "A2": [
        {
            "original": "Ein großer Vorteil der Ernährungspyramide: Sie ist vielseitig. Die Ernährungspyramide hilft Ihnen dabei. Es mangelt aber oftmals daran, dieses Wissen auch in ein entsprechendes Verhalten, eben in die alltägliche Ernährungspraxis umzusetzen.",
            "simplified": "Sie kann Ihnen helfen, gesund und mit Abwechslung zu essen. Die Pyramide ist eine Orientierungshilfe."
        },
        {
            "original": "Vom Koffeingehalt abgesehen bewerten Ernährungsexperten Energy-Drinks generell skeptisch - wegen ihres hohen Zuckergehaltes, teilweise aber auch wegen ihres Gehaltes an anderen typischen Zutaten wie Taurin, Glucuronolacton und Inosit, deren Wechselwirkungen untereinander und in Verbindung mit Alkohol noch nicht ganz klar sind.",
            "simplified": "Zu viel Koffein ist nicht gut für den Körper."
        },
        {
            "original": "Säureregulatoren (Kaliumlactat, Natriumacetate) sorgen dafür, dass das Produkt länger haltbar ist. Je nach Produkt kommen diverse Zusatzstoffe dazu wie Verdickungsmittel (z. B. Carrageen, Guarkernmehl, Johannisbrotkernmehl), Farbstoffe oder Aromen. Außer diesen pflanzlichen „Grundstoffen“ sorgen weitere Zutaten für die gewünschte Konsistenz, die Optik und den Geschmack, zum Beispiel Rapsöl, Sojaproteinkonzentrat (Wasser- und Fettbindemittel) und Sojaprotein-Isolat (Emulgator). Dafür verwenden sie unterschiedliche „Grundstoffe“.",
            "simplified": "Dafür müssen diese Produkte stark verarbeitet werden."
        }
    ],
    "B1": [
        {
            "original": "Ein großer Vorteil der Ernährungspyramide: Sie ist vielseitig. Die Ernährungspyramide hilft Ihnen dabei. Es mangelt aber oftmals daran, dieses Wissen auch in ein entsprechendes Verhalten, eben in die alltägliche Ernährungspraxis umzusetzen.",
            "simplified": "Ein großer Vorteil der Ernährungspyramide ist ihre Vielseitigkeit. Sie hilft Ihnen dabei. Jedoch fehlt es oft daran, dieses Wissen auch im Alltag umzusetzen, besonders bei der Ernährung."
        },
        {
            "original": "Vom Koffeingehalt abgesehen bewerten Ernährungsexperten Energy-Drinks generell skeptisch - wegen ihres hohen Zuckergehaltes, teilweise aber auch wegen ihres Gehaltes an anderen typischen Zutaten wie Taurin, Glucuronolacton und Inosit, deren Wechselwirkungen untereinander und in Verbindung mit Alkohol noch nicht ganz klar sind.",
            "simplified": "Abgesehen vom Koffeingehalt betrachten Ernährungsexperten Energy-Drinks im Allgemeinen skeptisch - wegen ihres hohen Zuckergehalts und manchmal auch wegen anderer üblicher Zutaten wie Taurin, Glucuronolacton und Inosit. Es ist noch nicht vollständig bekannt, wie diese Stoffe untereinander und in Kombination mit Alkohol wirken."
        },
        {
            "original": "Säureregulatoren (Kaliumlactat, Natriumacetate) sorgen dafür, dass das Produkt länger haltbar ist. Je nach Produkt kommen diverse Zusatzstoffe dazu wie Verdickungsmittel (z. B. Carrageen, Guarkernmehl, Johannisbrotkernmehl), Farbstoffe oder Aromen. Außer diesen pflanzlichen „Grundstoffen“ sorgen weitere Zutaten für die gewünschte Konsistenz, die Optik und den Geschmack, zum Beispiel Rapsöl, Sojaproteinkonzentrat (Wasser- und Fettbindemittel) und Sojaprotein-Isolat (Emulgator). Dafür verwenden sie unterschiedliche „Grundstoffe“.",
            "simplified": "Säureregulatoren wie Kaliumlactat und Natriumacetat sorgen dafür, dass das Produkt länger haltbar ist. Dazu kommen je nach Produkt verschiedene Zusatzstoffe wie Verdickungsmittel (z. B. Carrageen, Guarkernmehl, Johannisbrotkernmehl), Farbstoffe und Aromen. Zusätzlich zu diesen pflanzlichen Grundstoffen dienen weitere Zutaten wie Rapsöl, Sojaproteinkonzentrat (als Wasser- und Fettbindemittel) und Sojaprotein-Isolat (als Emulgator) der gewünschten Konsistenz, Optik und Geschmack des Produkts."
        }
    ],
    "B2": [
        {
            "original": "Ein großer Vorteil der Ernährungspyramide: Sie ist vielseitig. Die Ernährungspyramide hilft Ihnen dabei. Es mangelt aber oftmals daran, dieses Wissen auch in ein entsprechendes Verhalten, eben in die alltägliche Ernährungspraxis umzusetzen.",
            "simplified": "Die Ernährungspyramide bietet einen großen Vorteil durch ihre Vielseitigkeit und Unterstützung. Häufig fehlt es jedoch daran, dieses Wissen auch im Alltag praktisch umzusetzen, besonders bei der täglichen Ernährung."
        },
        {
            "original": "Vom Koffeingehalt abgesehen bewerten Ernährungsexperten Energy-Drinks generell skeptisch - wegen ihres hohen Zuckergehaltes, teilweise aber auch wegen ihres Gehaltes an anderen typischen Zutaten wie Taurin, Glucuronolacton und Inosit, deren Wechselwirkungen untereinander und in Verbindung mit Alkohol noch nicht ganz klar sind.",
            "simplified": "Ernährungsexperten betrachten Energy-Drinks generell kritisch, vor allem wegen ihres hohen Zuckergehalts und auch wegen anderer typischer Zutaten wie Taurin, Glucuronolacton und Inosit. Es ist noch nicht vollständig bekannt, wie sich diese Stoffe gegenseitig beeinflussen und in Kombination mit Alkohol wirken können."
        },
        {
            "original": "Säureregulatoren (Kaliumlactat, Natriumacetate) sorgen dafür, dass das Produkt länger haltbar ist. Je nach Produkt kommen diverse Zusatzstoffe dazu wie Verdickungsmittel (z. B. Carrageen, Guarkernmehl, Johannisbrotkernmehl), Farbstoffe oder Aromen. Außer diesen pflanzlichen „Grundstoffen“ sorgen weitere Zutaten für die gewünschte Konsistenz, die Optik und den Geschmack, zum Beispiel Rapsöl, Sojaproteinkonzentrat (Wasser- und Fettbindemittel) und Sojaprotein-Isolat (Emulgator). Dafür verwenden sie unterschiedliche „Grundstoffe“.",
            "simplified": "Säureregulatoren wie Kaliumlactat und Natriumacetate verlängern die Haltbarkeit des Produkts. Dazu kommen bei verschiedenen Produkten weitere Zusatzstoffe wie Verdickungsmittel (z. B. Carrageen, Guarkernmehl, Johannisbrotkernmehl), Farbstoffe und Aromen. Zusätzlich zu diesen pflanzlichen Grundstoffen tragen weitere Zutaten wie Rapsöl, Sojaproteinkonzentrat (als Wasser- und Fettbindemittel) und Sojaprotein-Isolat (als Emulgator) zur gewünschten Konsistenz, Optik und Geschmack bei."
        }
    ]
}

In [145]:
system_prompt = """
You are an assistant for simplifying text for non-native speakers of German.
"""

user_prompt = ''' 
Please rewrite the following complex sentence as language level {level} in order to make it easier 
to understand by non-native speakers of German. 
You can do so by replacing complex words with simpler synonyms (i.e. paraphrasing), 
deleting unimportant information (i.e. compression), 
and/or splitting a long complex sentence into several simpler ones. Shorten the output to make it more understandable.
The final simplified sentence needs to be grammatical, fluent, and retain the main ideas 
of its original counterpart without altering its meaning. 


Examples of how to do it:

Complex sentence: {original_1}
Simplified sentence: {simpl_1}

Complex sentence: {original_2}
Simplified sentence: {simpl_2}

Complex sentence: {original_3}
Simplified sentence: {simpl_3}

And now the complex to be simplified:
Complex sentence: {original_sent} 
Simplified sentence:
'''

prompt_template = "<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n<|im_start|>assistant\n"

In [146]:
def run_model_sents(input_sentence, params = {"max_new_tokens": 2048}):
     
    examples = level_examples[level]
    
    example_1_complex = examples[0]['original']
    example_1_simple = examples[0]['simplified']
    
    example_2_complex = examples[1]['original']
    example_2_simple = examples[1]['simplified']
    
    example_3_complex = examples[2]['original']
    example_3_simple = examples[2]['simplified']
        
        
    input_user = user_prompt.format(original_1=example_1_complex, simpl_1=example_1_simple,
                                    original_2=example_2_complex, simpl_2=example_2_simple,
                                    original_3=example_3_complex, simpl_3=example_3_simple,
                                    original_sent=input_sentence, level = level)
    
    prompt = prompt_template.format(system_prompt=system_prompt, user_prompt=input_user)
    #print(prompt)
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    outputs = model.generate(inputs.to("cuda:0"), 
                                        pad_token_id=tokenizer.eos_token_id,
                                        **params) #tur_inputs.to("cuda"), tur_inputs.to("cuda:0")
    
    decoded_outputs = tokenizer.decode(outputs[0][inputs.shape[1]:],
                                       skip_special_tokens=True) 
    
    return decoded_outputs

In [147]:
level='B2'
answer_simplified = run_model_sents(answer)
print(answer_simplified)

Wenn Ihr Blutdruckmessgerät nicht funktioniert, können Sie Ihren Blutdruck zu Hause manuell messen. Hier ist wie:
1. Suchen Sie eine gute Lichtquelle, damit Sie Ihren Arm und Ihren Finger gut sehen können.
2. Waschen Sie Ihre Hände und Ihren Arm und trocknen Sie sie ab.
3. Finden Sie eine Vene an Ihrem Arm, die flach liegt (d. h. nicht hervorsteht). Das ist wichtig, weil es dem Stethoskop hilft, den Puls zu hören.
4. Legen Sie Ihren Finger unterhalb und parallel zum Handgelenkknochen.
5. Drücken Sie Ihren Finger gegen den Handgelenkknochen, bis kein Puls mehr zu spüren ist. Dies ist der obere Bereich des systolischen Drucks.
6. Lassen Sie Ihren Finger los, und Ihr Blut wird durch Ihren Arm fließen. Spüren Sie den Puls in Ihrem Finger oder an Ihrem Handgelenk. Dies ist der untere Bereich des diastolischen Drucks.
7. Wiederholen Sie dies mindestens zweimal und notieren Sie jedes Mal die Werte.
8. Notieren Sie die Werte und notieren Sie das Datum.
Es ist wichtig zu beachten, dass diese Me

In [148]:
level='B1'
answer_simplified = run_model_sents(answer)
print(answer_simplified)

Antwort: Wenn Ihr Blutdruckmessgerät nicht funktioniert, können Sie eine manuelle Blutdruckmessung zu Hause durchführen. Es gibt verschiedene Möglichkeiten, den Blutdruck zu messen, aber die häufigste Methode ist die Verwendung eines Blutdruckmessgeräts. Wenn Sie Ihr Gerät nicht verwenden können, sollten Sie einen Arzt aufsuchen, um Ihren Blutdruck zu überprüfen. Wenn Sie jedoch Ihren Blutdruck zu Hause manuell messen müssen, befolgen Sie diese Schritte:

1. Suchen Sie eine gute Lichtquelle, damit Sie Ihren Arm und Ihren Finger gut sehen können.
2. Waschen Sie Ihre Hände und Ihren Arm und trocknen Sie sie ab.
3. Finden Sie eine Vene an Ihrem Arm, die flach liegt (d. h. nicht hervorsteht). Das ist wichtig, weil es dem Stethoskop hilft, den Puls zu hören.
4. Legen Sie Ihren Finger unterhalb und parallel zum Handgelenkknochen.
5. Drücken Sie Ihren Finger gegen den Handgelenkknochen, bis kein Puls mehr zu spüren ist. Dies ist der obere Bereich des systolischen Drucks.
6. Lassen Sie Ihren F

In [149]:
# Iterate through 'original' column
level='A2'
answer_simplified = run_model_sents(answer)
print(answer_simplified)

Wenn Ihr Blutdruckmessgerät nicht funktioniert, können Sie Ihren Blutdruck zu Hause manuell messen. Hier ist wie:
1. Suchen Sie eine gute Lichtquelle, damit Sie Ihren Arm und Ihren Finger gut sehen können.
2. Waschen Sie Ihre Hände und Ihren Arm und trocknen Sie sie ab.
3. Finden Sie eine Vene an Ihrem Arm, die flach liegt (d. h. nicht hervorsteht). Das ist wichtig, weil es dem Stethoskop hilft, den Puls zu hören.
4. Legen Sie Ihren Finger unterhalb und parallel zum Handgelenkknochen.
5. Drücken Sie Ihren Finger gegen den Handgelenkknochen, bis kein Puls mehr zu spüren ist. Dies ist der obere Bereich des systolischen Drucks.
6. Lassen Sie Ihren Finger los, und Ihr Blut wird durch Ihren Arm fließen. Spüren Sie den Puls in Ihrem Finger oder an Ihrem Handgelenk. Dies ist der untere Bereich des diastolischen Drucks.
7. Wiederholen Sie dies mindestens zweimal und notieren Sie jedes Mal die Werte.
8. Notieren Sie die Werte und notieren Sie das Datum.
Es ist wichtig zu beachten, dass diese Me

In [150]:
level='A1'
answer_simplified = run_model_sents(answer)
print(answer_simplified)

Antwort:
Wenn Ihr Blutdruckmessgerät nicht funktioniert, können Sie Ihren Blutdruck zu Hause manuell messen. Hier ist wie:

1. Finden Sie eine Vene an Ihrem Arm, die flach liegt (d. h. nicht hervorsteht). Das ist wichtig, weil es dem Stethoskop hilft, den Puls zu hören.
2. Waschen Sie Ihre Hände und Ihren Arm und trocknen Sie sie ab.
3. Legen Sie Ihren Finger unterhalb und parallel zum Handgelenkknochen.
4. Drücken Sie Ihren Finger gegen den Handgelenkknochen, bis kein Puls mehr zu spüren ist. Dies ist der obere Bereich des systolischen Drucks.
5. Lassen Sie Ihren Finger los, und Ihr Blut wird durch Ihren Arm fließen. Spüren Sie den Puls in Ihrem Finger oder an Ihrem Handgelenk. Dies ist der untere Bereich des diastolischen Drucks.
6. Wiederholen Sie dies mindestens zweimal und notieren Sie jedes Mal die Werte.
7. Notieren Sie die Werte und notieren Sie das Datum.

Es ist wichtig zu beachten, dass diese Methode nicht so genau wie die Verwendung eines Blutdruckmessgeräts ist. Wenn mögli